- todo liste:
    - tester la construction des images via le fichier setup.sh  👍🏼 
    - le lancement des images via docker-compose  +  datascience api  👍🏼     
        - Je n'arrive pas à lancer l'api datascientest via docker compose. (Elle se lance bien avec le dockercompose, mais il ne semble pas atteignable )
         je ne suis pas certain que les containers soient sur les mêmes réseaux car il y a création d'un "docker_eval_my_network"   

        - j'ajoute juste les volumes.  ✍🏼.
        ERROR: for authentication  Cannot create container for service authentication: failed to mount local volume: mount /home/ubuntu/docker_volumes:/var/lib/docker/volumes/docker_eval_my_test_volume/_data, flags: 0x1000: no such file or directory

        ERROR: for authentication  Cannot create container for service authentication: failed to mount local volume: mount /home/docker_volumes:/var/lib/docker/volumes/docker_eval_my_test_volume/_data, flags: 0x1000: no such file or directory

        >> Les containers Authorzation et content n'ont pas été créé, le fichier api_test.log n'est pas aparu dans docker volumes
        

        >> il y a un souci dans le docker compose, je n'arricve pas aà crer les container avec les noms 

        ERROR: The image for the service you're trying to recreate has been removed. If you continue, volume data could be lost. Consider backing up your data before continuing.

        ERROR: pull access denied for content, repository does not exist or may require 'docker login': denied: requested access to the resource is denied

        -- j'eassaie avec ka creation desu volume avec le setup, puis ensuite avec seulement docker compose

        Il semble que le bridge du volume ne fonctionne pas. 

            - il n'y avait pas l'execution du script python, je les avait laissé en commentaire
        - Il semble que je ne puisse pas requeter en construisant les images et en les lançent directement 
        

    - voir si les testes produits par les containers fonctionnent bien  👍🏼
        - les volumes pour intéger le user.csv à la fois dans les containers et dans le dossier source
    - envoyer les resultats des tests dans un fichier  log.txt à la racine du dossier 👍🏼
    - faire la documentation ( remarque sur les choix effectés)
    - mettre les images dans dockerhub


    #### Rendu 
    
    - ✅ un fichier docker-compose.yml qui contient l'enchaînement des tests à effectuer
    - ❌ les fichiers Python utilisés dans les images Docker
    - les fichiers Dockerfile utilisés pour construire ces images
    - un fichier de appelé setup.sh contenant les commandes utilisées pour construire les images et lancer le docker-compose
    - le résultat des logs dans un fichier log.txt
    - éventuellement un fichier de remarques ou de justification des choix effectués


In [9]:
#-------------------------------------Authentication
import os
import requests
import csv

# définition de l'adresse de l'API
address = '34.244.150.51'
# port de l'API
port = 8000

def request_test(user, password):
    # requête
    r = requests.get(
        url=f'http://{address}:{port}/permissions',
        params= {
            'username': user,
            'password': password
        }
    )

    output = '''
    ============================
        Authentication test
    ============================

    request done at "/permissions"
    | username={user}
    | password={password}

    expected result = 200
    actual restult = {status_code}

    ==>  {test_status}

    '''

    # statut de la requête
    status_code = r.status_code

    # affichage des résultats
    if status_code == 200:
        test_status = 'SUCCESS'
    else:
        test_status = 'FAILURE'
    log=output.format(status_code=status_code, test_status=test_status, user=user, password=password)

    # impression dans un fichier
    with open('api_test.log', 'a') as file:
        file.write(log)

with open('user.csv', newline='') as f:
    reader = csv.DictReader(f)
    users = list(reader)

for user in users:
    request_test(user['users'], user['password'])


In [2]:
#-------------------------------------Authorization
import os
import requests
import csv

# définition de l'adresse de l'API
address = '34.244.150.51'
# port de l'API
port = 8000

def request_test(user, password,version):
    # requête
    r = requests.get(
        url=f'http://{address}:{port}/{version}/sentiment',
        params= {
            'username': user,
            'password': password, 
            'sentence': "life is beautiful"
        }
    )

    output = '''
    ============================
        Authentication test
    ============================

    request done at "{version}/sentiment"
    | username={user}
    | password={password}

    expected result = 200
    actual restult = {status_code}

    ==>  {test_status}

    '''

    # statut de la requête
    status_code = r.status_code

    # affichage des résultats
    if status_code == 200:
        test_status = 'SUCCESS'
    else:
        test_status = 'FAILURE'
    print(output.format(status_code=status_code, test_status=test_status, user=user, 
    password=password, version=version))

    # impression dans un fichier
    if os.environ.get('LOG') == 1:
        with open('api_test.log', 'a') as file:
            file.write(output)

with open('user.csv', newline='') as f:
    reader = csv.DictReader(f)
    users = list(reader)

versions=["v1", "v2"]
for user in users:
    if not user['users']=="clementine":
        for version in versions:
            request_test(user['users'], user['password'], version)


        Authentication test

    request done at "v1/sentiment"
    | username=alice
    | password=wonderland

    expected result = 200
    actual restult = 200

    ==>  SUCCESS

    

        Authentication test

    request done at "v2/sentiment"
    | username=alice
    | password=wonderland

    expected result = 200
    actual restult = 200

    ==>  SUCCESS

    

        Authentication test

    request done at "v1/sentiment"
    | username=bob
    | password=builder

    expected result = 200
    actual restult = 200

    ==>  SUCCESS

    

        Authentication test

    request done at "v2/sentiment"
    | username=bob
    | password=builder

    expected result = 200
    actual restult = 403

    ==>  FAILURE

    


In [3]:
#-------------------------------------Content

import os
import requests
import csv

# définition de l'adresse de l'API
address = '34.244.150.51'
# port de l'API
port = 8000

def request_test(user, password,version, sentence):
    # requête
    r = requests.get(
        url=f'http://{address}:{port}/{version}/sentiment',
        params= {
            'username': user,
            'password': password, 
            'sentence': sentence
        }
    )

    output = '''
    ============================
        Authentication test
    ============================

    request done at "{version}/sentiment"
    | username={user}
    | password={password}
    | sentence={sentence}
    | score = {r}

    expected result = 200
    actual restult = {status_code}

    ==>  {test_status}

    '''

    # statut de la requête
    status_code = r.status_code

    # affichage des résultats
    if status_code == 200:
        test_status = 'SUCCESS'
    else:
        test_status = 'FAILURE'
    print(output.format(status_code=status_code, test_status=test_status, user=user, 
    password=password, version=version, sentence=sentence, r=r.json()['score']))

    # impression dans un fichier
    if os.environ.get('LOG') == 1:
        with open('api_test.log', 'a') as file:
            file.write(output)

with open('user.csv', newline='') as f:
    reader = csv.DictReader(f)
    users = list(reader)

versions=["v1", "v2"]
sentences=["life is beautiful", "that sucks" ]
for user in users:
    if not user['users'] in ["clementine","bob"]:
        for version in versions:
            for sentence in sentences:
                request_test(user['users'], user['password'], version, sentence)


        Authentication test

    request done at "v1/sentiment"
    | username=alice
    | password=wonderland
    | sentence=life is beautiful
    | score = 0.5994

    expected result = 200
    actual restult = 200

    ==>  SUCCESS

    

        Authentication test

    request done at "v1/sentiment"
    | username=alice
    | password=wonderland
    | sentence=that sucks
    | score = -0.3612

    expected result = 200
    actual restult = 200

    ==>  SUCCESS

    

        Authentication test

    request done at "v2/sentiment"
    | username=alice
    | password=wonderland
    | sentence=life is beautiful
    | score = 0.5994

    expected result = 200
    actual restult = 200

    ==>  SUCCESS

    

        Authentication test

    request done at "v2/sentiment"
    | username=alice
    | password=wonderland
    | sentence=that sucks
    | score = -0.3612

    expected result = 200
    actual restult = 200

    ==>  SUCCESS

    
